https://github.com/prdeepakbabu/Python/blob/master/Deep%20learning%20gender/Deep%20Learning%20(RNN%20-%20LSTMs)%20Predict%20Gender%20from%20Name.ipynb 

https://github.com/mhjabreel/CharCnn_Keras/blob/master/models/char_cnn_zhang.py 

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.keras.layers.core import Dense, Activation, Dropout
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense, Embedding
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Input, Dense, Flatten
from tensorflow.python.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"

In [5]:
%cd $PATH_TO_FOLDER

/content/drive/MyDrive/Name2Demographics


In [3]:
%cd ../../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [5]:
import sys  
sys.path.insert(0, 'Models/PreProcessing/')

from char_utils import *

sys.path.insert(0, 'Models/AIEEEGenderData/Preprocessing/')
from aieee_gender import *

In [7]:
# print(tf.__version__)

In [8]:
# physical_devices = tf.config.list_physical_devices('GPU')
# # tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# from tensorflow.python.client import device_lib

# def get_available_devices():
#     local_device_protos = device_lib.list_local_devices()
#     return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']

In [9]:
# get_available_devices()

### PreProcessing

In [6]:
aieee_df = AIEEEGenderData()

In [7]:
train_x, train_y, val_x, val_y, test_x, test_y = splitData(aieee_df, 0.7, 0, 'Name', 'char_lstm_cbse_oov.txt')

(train:(284697, 4), val:(0, 0), test:(124589, 4))
Train: ((278492, 30),(278492, 2)) Val:((0,),(0,)) Test: ((122434, 30),(122434, 2))


In [8]:
voc = vocab()

### Build Model

In [ ]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

In [9]:
inputs = Input(MAXLEN_NAME_CHAR_LSTM)
x = Embedding(len(voc), len(voc))(inputs)
# x = LSTM(64, return_sequences=True, activation='tanh')(x)
# x = Dropout(0.2)(x)
x = LSTM(64, return_sequences=False, activation='tanh')(x)
x = Dropout(0.2)(x)
# x = Dense(8, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

2022-03-17 14:51:43.354260: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-17 14:51:43.354401: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: obelix
2022-03-17 14:51:43.354431: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: obelix
2022-03-17 14:51:43.354894: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-03-17 14:51:43.363108: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-03-17 14:51:43.363137: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.86.0
2022-03-17 14:51:43.376621: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 28)            784       
_________________________________________________________________
lstm (LSTM)                  (None, 64)                23808     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 24,722
Trainable params: 24,722
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [12]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [13]:
batch_size=100

In [14]:
model.fit(train_x, train_y,batch_size=batch_size,epochs=10,validation_data=(test_x, test_y))

Epoch 1/10
2785/2785 [==============================] - 148s 51ms/step - loss: 0.4599 - accuracy: 0.7786 - val_loss: 0.3809 - val_accuracy: 0.8402
Epoch 2/10
2785/2785 [==============================] - 139s 50ms/step - loss: 0.2984 - accuracy: 0.8776 - val_loss: 0.3079 - val_accuracy: 0.8719
Epoch 3/10
2785/2785 [==============================] - 137s 49ms/step - loss: 0.2391 - accuracy: 0.9059 - val_loss: 0.2626 - val_accuracy: 0.8921
Epoch 4/10
2785/2785 [==============================] - 136s 49ms/step - loss: 0.2020 - accuracy: 0.9228 - val_loss: 0.2424 - val_accuracy: 0.9010
Epoch 5/10
2785/2785 [==============================] - 136s 49ms/step - loss: 0.1619 - accuracy: 0.9394 - val_loss: 0.2194 - val_accuracy: 0.9184
Epoch 7/10
2785/2785 [==============================] - 137s 49ms/step - loss: 0.1501 - accuracy: 0.9438 - val_loss: 0.2111 - val_accuracy: 0.9184
Epoch 8/10
2785/2785 [==============================] - 136s 49ms/step - loss: 0.1406 - accuracy: 0.9474 - val_loss: 0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
score, acc = model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

3827/3827 [==============================] - 31s 8ms/step - loss: 0.2085 - accuracy: 0.9253
Test score: 0.2085089534521103
Test accuracy: 0.9253230094909668


In [16]:
pred = model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male     0.9123    0.9646    0.9377     71377
      Female     0.9462    0.8704    0.9067     51057

    accuracy                         0.9253    122434
   macro avg     0.9293    0.9175    0.9222    122434
weighted avg     0.9265    0.9253    0.9248    122434

--------------------------------------------------------


In [21]:
filename = "Models/AIEEEGenderData/CharLSTM/SavedModel/CharLSTM"
model.save(filename)

INFO:tensorflow:Assets written to: Models/CBSEData/CharLSTM/SavedModel/CharLSTM/assets
